In [28]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from read_HiC import name_chromosomes

## BIN NUCLEAR RNAseq DATA FOR GENE ABUNDANCE 

In this notebook we read processed nuclear RNAseq data from Stevens et al. (3D structures of individual mammalian genomes studied by single-cell Hi-C) and we bin it in order to couple it with topological information with the same resolution.

Load gene data

In [22]:
# Load data
data_abundance = pd.read_csv('/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/data/GSE117765_matrix_PEO1_tpm.csv', sep=',')
chrom_info = pd.read_csv('/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/data/hg38_genesanno.csv', sep=',')
data_abundance

,gene,PEO1_Adherent_1,PEO1_Adherent_2,PEO1_Clone10,PEO1_Clone18,PEO1_Clone4,PEO1_Clone5
0,C14orf39,0.689563,0.847225,0.725268,0.409947,0.892770,0.369792
1,LMNB1,85.463505,83.182113,52.882031,51.027912,49.247563,49.451937
2,SLC4A7,9.586560,9.645629,8.519439,7.876676,6.731346,7.983751
3,DLGAP4,21.301878,16.044113,21.521601,28.264900,27.975815,26.079266
4,C4orf19,1.658426,1.364189,2.373173,2.789502,2.713487,3.388334
...,...,...,...,...,...,...,...
17867,RP11-47I22.3,0.059052,0.038399,0.041177,0.129289,0.072731,0.071309
17868,ACOT7,20.747591,22.342015,24.039646,23.172762,23.591160,23.921450
17869,SNURF,36.591841,34.334813,36.890917,48.547608,41.593939,47.253662
17870,VPS4A,34.079362,36.997386,40.946816,41.520269,38.111980,45.131026


In [23]:
# Select relevant columns from chrom_info
info = {
    'gene': chrom_info['gene_name'],
    'start position': chrom_info['start_position'],
    'end position': chrom_info['end_position'],
    'chromosome': chrom_info['chromosome_name'],
    'strand': chrom_info['strand']
}

info_df = pd.DataFrame(info)
info_df

,gene,start position,end position,chromosome,strand
0,DDX11L17,182696,184174,1,+
1,PRDM16,3069168,3438621,1,+
2,PEX10,2403964,2413797,1,-
3,RPL21P21,10054445,10054781,1,-
4,LINC01345,3944547,3949024,1,-
...,...,...,...,...,...
42338,MT-ND6,14149,14673,MT,-
42339,MT-TE,14674,14742,MT,-
42340,MT-CYB,14747,15887,MT,+
42341,MT-TT,15888,15953,MT,+


In [24]:
# Merge the two datasets
RNA_seq = pd.merge(data_abundance, info_df, on='gene')
RNA_seq 

,gene,PEO1_Adherent_1,PEO1_Adherent_2,PEO1_Clone10,PEO1_Clone18,PEO1_Clone4,PEO1_Clone5,start position,end position,chromosome,strand
0,C14orf39,0.689563,0.847225,0.725268,0.409947,0.892770,0.369792,60396469,60515543,14,-
1,LMNB1,85.463505,83.182113,52.882031,51.027912,49.247563,49.451937,126776623,126837020,5,+
2,SLC4A7,9.586560,9.645629,8.519439,7.876676,6.731346,7.983751,27372721,27484420,3,-
3,DLGAP4,21.301878,16.044113,21.521601,28.264900,27.975815,26.079266,36306336,36528637,20,+
4,C4orf19,1.658426,1.364189,2.373173,2.789502,2.713487,3.388334,37453925,37623495,4,+
...,...,...,...,...,...,...,...,...,...,...,...
15895,SSBP4,35.709370,37.520894,28.588256,26.623834,24.951856,28.293333,18418864,18434562,19,+
15896,TMEM17,0.830824,0.795552,0.648648,0.632519,0.714249,0.734691,62500218,62511894,2,-
15897,ACOT7,20.747591,22.342015,24.039646,23.172762,23.591160,23.921450,6264269,6393767,1,-
15898,SNURF,36.591841,34.334813,36.890917,48.547608,41.593939,47.253662,24954987,24977850,15,+


Set bin sizes (in bp units). Recommended: 1Mb (1000000)

In [25]:
chrom_number=23
resolution = 1000000

chr_vec=name_chromosomes(chrom_number)
chr_vec=['{}'.format(chrom[3:]) for chrom in chr_vec]
length_chromosomes = pd.read_csv('/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/data/length_chromosomes_Mb_Hs.txt', header=None)
length_chromosomes = np.array(length_chromosomes.iloc[0])+ np.ones(chrom_number)
#Convert length from Mb to bp
length_chromosomes = length_chromosomes* np.ones(len(length_chromosomes))* 1000000

Bin abundance data

In [36]:
for t, chrom in enumerate(chr_vec):
    Abundance_chrom = RNA_seq[RNA_seq['chromosome']==chrom]
    
    #Find genes middle point
    length = (Abundance_chrom['end position'] -  Abundance_chrom['start position'])//2
    Abundance_chrom['position']=  Abundance_chrom['start position'] + length
    Abundance_chrom = Abundance_chrom.sort_values(by=['position'])
    
    #Create bins
    end = length_chromosomes[t]
    bins=np.arange(0, end, resolution)
    n_bins=len(bins)
    
    #Bin abundance
    Abundance_binned=np.zeros(n_bins)
    for j in range(n_bins-1):
        select_chr= Abundance_chrom[(Abundance_chrom['position']< bins[j+1]) & (Abundance_chrom['position']> bins[j])]
        Abundance_binned[j]=len(select_chr['gene'])
        
    # Save binned abundance
    output_directory = '/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/data/rna data/binned abundance/resolution 1Mb/'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    df_bin_abundance={'Abundance': Abundance_binned, 'Bins': bins}
    df_bin_abundance=pd.DataFrame(df_bin_abundance)
    df_bin_abundance.to_csv('/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/data/rna data/binned abundance/resolution 1Mb/'\
                            'abundance_chr{}.csv'.format(chrom))